In [9]:
import numpy as np
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [10]:
y_name = ['ccn_001',
 'ccn_003',
 'ccn_006',
 'CHI',
 'CHI_CCN',
 'D_ALPHA',
 'D_GAMMA',
 'D_ALPHA_CCN',
 'D_GAMMA_CCN',
 'PM25']

In [11]:
x_name = ['TOT_NUM_CONC',
 'TOT_MASS_CONC',
 'pmc_SO4',
 'pmc_NO3',
 'pmc_Cl',
 'pmc_NH4',
 'pmc_ARO1',
 'pmc_ARO2',
 'pmc_ALK1',
 'pmc_OLE1',
 'pmc_API1',
 'pmc_API2',
 'pmc_LIM1',
 'pmc_LIM2',
 'pmc_OC',
 'pmc_BC',
 'pmc_H2O',
 'TEMPERATURE',
 'REL_HUMID',
 'ALT',
 'Z',
 'XLAT',
 'XLONG',
 'h2so4',
 'hno3',
 'hcl',
 'nh3',
 'no',
 'no2',
 'no3',
 'n2o5',
 'hono',
 'hno4',
 'o3',
 'o1d',
 'O3P',
 'oh',
 'ho2',
 'h2o2',
 'co',
 'so2',
 'ch4',
 'c2h6',
 'ch3o2',
 'ethp',
 'hcho',
 'ch3oh',
 'ANOL',
 'ch3ooh',
 'ETHOOH',
 'ald2',
 'hcooh',
 'RCOOH',
 'c2o3',
 'pan',
 'aro1',
 'aro2',
 'alk1',
 'ole1',
 'api1',
 'api2',
 'lim1',
 'lim2',
 'par',
 'AONE',
 'mgly',
 'eth',
 'OLET',
 'OLEI',
 'tol',
 'xyl',
 'cres',
 'to2',
 'cro',
 'open',
 'onit',
 'rooh',
 'ro2',
 'ano2',
 'nap',
 'xo2',
 'xpar',
 'isop',
 'isoprd',
 'isopp',
 'isopn',
 'isopo2',
 'api',
 'lim',
 'dms',
 'msa',
 'dmso',
 'dmso2',
 'ch3so2h',
 'ch3sch2oo',
 'ch3so2',
 'ch3so3',
 'ch3so2oo',
 'ch3so2ch2oo',
 'SULFHOX',
 'P',
 'PB',
 'cos(Time)']

In [27]:
targ_norm_4d = np.zeros((10, 39, 133, 159, 169)) * np.nan
targ_norm = np.zeros((10, 39*133*159*169)) * np.nan
targ_mean = np.zeros((10, 39)) * np.nan
targ_std = np.zeros((10, 39)) * np.nan

for iy in range(len(y_name)):
    for iz in range(39):
        ds = xr.open_dataset("/home/chuchun2/hackathon_data/training.nc").sel(bottom_top=iz)

        y = ds[y_name[iy]]  #.values.flatten()
        y_max = y.max()
        y_min = np.where(y == 0, y_max, y).min()  # min of the non-zero values
        y_new = np.where(y == 0, y_min, y)  # replace 0 with min of the non-zero values
        y_log = np.log(y_new)
        targ_mean[iy, iz] = np.mean(y_log)
        targ_std[iy, iz] = y_log.std()
        targ_norm_4d[iy, iz, :, :, :] = (y_log - targ_mean[iy, iz]) / targ_std[iy, iz]
    
    targ_norm[iy, :] = targ_norm_4d[iy, :, :, :].flatten()
    

In [37]:
# plt.hist(targ_norm_4d[:, 0, :, :].flatten())

In [15]:
# 102 features + cos(time)
# feat_norm_4d = np.zeros((103, 39, 133, 159, 169)) * np.nan
# feat_norm = np.zeros((103, 39*133*159*169)) * np.nan
feat_mean = np.zeros((103, 39)) * np.nan
feat_std = np.zeros((103, 39)) * np.nan


for iz in range(39):
    ds = xr.open_dataset("/home/chuchun2/hackathon_data/training.nc").sel(bottom_top=iz)
    for ix in range(len(x_name)-3):
        if (x_name[ix]!='Z'):
            x = ds[x_name[ix]]  #.values.flatten()
            x_max = x.max()
            x_min = np.where(x == 0, x_max, x).min()
            x_new = np.where(x == 0, x_min, x)
            x_log = np.log(x_new)
            feat_mean[ix, iz] = np.mean(x_log)
            feat_std[ix, iz] = x_log.std()
        else:
            x = ds[x_name[ix]]  #.values.flatten()
            x_max = x.max()
            x_min = np.where(x == 0, x_max, x).min()
            x_new = np.where(x == 0, x_min, x)
#             x_log = np.log(x_new)
            feat_mean[ix, iz] = np.mean(x_new)
            feat_std[ix, iz] = x_new.std()  
    for ix in [len(x_name)-3]:
        x = ds['P'] + ds['PB']  #.values.flatten()
        x_max = x.max()
        x_min = np.where(x == 0, x_max, x).min()
        x_new = np.where(x == 0, x_min, x)
        x_log = np.log(x_new)
        feat_mean[ix, iz] = np.mean(x_log)
        feat_std[ix, iz] = x_log.std()
    for ix in [len(x_name)-2]:
        x = ds['PB']  #.values.flatten()
        x_max = x.max()
        x_min = np.where(x == 0, x_max, x).min()
        x_new = np.where(x == 0, x_min, x)
        x_log = np.log(x_new)
        feat_mean[ix, iz] = np.mean(x_log)
        feat_std[ix, iz] = x_log.std()
        
#         feat_norm_4d[ix, iz, :, :, :] = (x_log - feat_mean[ix, iz]) / feat_std[ix, iz]
    
#     feat_norm[ix, :] = feat_norm_4d[ix, :, :, :].flatten()
    

/tmp/ipykernel_1215325/280360267.py:16: RuntimeWarning: invalid value encountered in log
  x_log = np.log(x_new)
/tmp/ipykernel_1215325/280360267.py:16: RuntimeWarning: invalid value encountered in log
  x_log = np.log(x_new)
/tmp/ipykernel_1215325/280360267.py:16: RuntimeWarning: invalid value encountered in log
  x_log = np.log(x_new)
/tmp/ipykernel_1215325/280360267.py:16: RuntimeWarning: invalid value encountered in log
  x_log = np.log(x_new)
/tmp/ipykernel_1215325/280360267.py:16: RuntimeWarning: invalid value encountered in log
  x_log = np.log(x_new)
/tmp/ipykernel_1215325/280360267.py:16: RuntimeWarning: invalid value encountered in log
  x_log = np.log(x_new)
/tmp/ipykernel_1215325/280360267.py:16: RuntimeWarning: invalid value encountered in log
  x_log = np.log(x_new)
/tmp/ipykernel_1215325/280360267.py:16: RuntimeWarning: invalid value encountered in log
  x_log = np.log(x_new)
/tmp/ipykernel_1215325/280360267.py:16: RuntimeWarning: invalid value encountered in log
  x_log

In [6]:
feat_mean.shape

(103, 39)

In [7]:
# np.save('./data/feat_mean.npy', feat_mean)

In [8]:
# np.save('./data/feat_std.npy', feat_std)